# Projet Knowledge Extraction - Partie A : Preprocessing et Représentation Text

**Université Paris Cité - Master 2 VMI**  
**Cours :** IFLCE085 Recherche et extraction sémantique à partir de texte (Prof. Salima Benbernou)

**Équipe :**
- **Partie A (Preprocessing) : Jacques Gastebois**
- Partie B : Boutayna EL MOUJAOUID
- Partie C : Franz Dervis
- Partie D : Aya Benkabour

---

## Dataset : NER (Named Entity Recognition)

Ce notebook traite un dataset de **2221 phrases** annotées pour la reconnaissance d'entités nommées.

**Colonnes originales :**
- `id` : Identifiant unique de la phrase
- `words` : Liste des mots tokenisés
- `ner_tags` : Tags NER (0=O, 1=B-LOC, 2=B-PER, 4=B-ORG)
- `text` : Texte brut de la phrase

**Colonnes ajoutées par ce notebook :**
- `cleaned_text` : Texte nettoyé (lowercase, sans caractères spéciaux)
- `lemmatized_text` : Texte lemmatisé avec spaCy

## Étape 1 : Setup et Importations

In [14]:
import sys
# Installation des dépendances de base si nécessaire
# - pandas : manipulation de données (DataFrames)
# - numpy : calcul numérique
# - nltk : traitement du langage naturel (non utilisé ici mais souvent requis)
# - scikit-learn : outils de machine learning
# - spacy : traitement avancé du langage naturel (lemmatization, NER, etc.)
!{sys.executable} -m pip install -q pandas numpy nltk scikit-learn spacy

# Téléchargement du modèle de langue anglais pour spaCy (version 'small')
!{sys.executable} -m spacy download en_core_web_sm

import os
import re
import pandas as pd
import numpy as np
import spacy

# Chargement du modèle spaCy 'en_core_web_sm'
# Ce modèle contient les règles de tokenization, lemmatization et les vecteurs de base pour l'anglais.
nlp = spacy.load('en_core_web_sm')

# Configuration de l'affichage pandas pour voir plus de contenu dans les colonnes
pd.set_option('display.max_colwidth', 100)

print("✅ Environnement configuré avec succès.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 17.9 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✅ Environnement configuré avec succès.


## Étape 2 : Chargement des Données

In [15]:
# Chargement du dataset depuis le fichier CSV
# Le fichier doit contenir les colonnes 'id', 'words', 'ner_tags', 'text'
df = pd.read_csv('data.csv')

# Affichage des informations de base sur le dataset chargé
print(f"📊 Dataset chargé : {len(df)} phrases")
print(f"\nColonnes originales : {list(df.columns)}")
print(f"\nAperçu des données :")

# Affichage des 5 premières lignes pour vérification
df.head()

📊 Dataset chargé : 700 phrases

Colonnes originales : ['id', 'words', 'ner_tags', 'text']

Aperçu des données :


,id,words,ner_tags,text
0,en-doc5809-sent11,"['When' 'Aeneas' 'later' 'traveled' 'to' 'Hades' ',' 'he' 'called' 'to'\n 'her' 'ghost' 'but' 's...",[0 2 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],"When Aeneas later traveled to Hades , he called to her ghost but she neither spoke to nor acknow..."
1,en-doc6123-sent45,['On' '23' 'November' '1969' 'he' 'wrote' 'to' 'The' 'Times' 'newspaper'\n 'saying' 'that' 'the'...,[0 0 0 0 0 0 0 4 4 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0],On 23 November 1969 he wrote to The Times newspaper saying that the preparation for show trials ...
2,en-doc5831-sent40,"['Stephenson' ""'s"" 'estimates' 'and' 'organising' 'ability' 'proved' 'to'\n 'be' 'inferior' 'to'...",[2 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 2 0 0 0],Stephenson 's estimates and organising ability proved to be inferior to those of Locke and the b...
3,en-doc6189-sent73,['France' 'then' 'postponed' 'a' 'visit' 'by' 'Sharon' '.'],[1 0 0 0 0 0 2 0],France then postponed a visit by Sharon .
4,en-doc6139-sent18,"['Only' 'twenty-seven' 'years' 'old' 'at' 'his' 'death' ',' 'Moseley'\n 'could' 'in' 'many' 'sci...",[0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],"Only twenty-seven years old at his death , Moseley could in many scientists ' opinions have cont..."


## Étape 3 : Nettoyage du Texte

Ajout de la colonne `cleaned_text` au dataset.

In [16]:
def clean_text(text):
    """
    Nettoie le texte pour standardiser son format.
    
    Args:
        text (str): La chaîne de caractères à nettoyer.
        
    Returns:
        str: Le texte nettoyé.
    """
    # Vérification que l'entrée est bien une chaîne de caractères
    if not isinstance(text, str):
        return ""
    
    # 1. Conversion en minuscules (Lowercase)
    # Cela permet de traiter 'Apple' et 'apple' de la même manière (réduction de vocabulaire).
    text = text.lower()
    
    # 2. Suppression des caractères spéciaux
    # On ne garde que les caractères alphanumériques (a-z, 0-9) et les espaces (\s).
    # Tout le reste (ponctuation, symboles) est remplacé par un espace.
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    
    # 3. Normalisation des espaces
    # On remplace les suites d'espaces par un seul espace et on supprime les espaces en début/fin.
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Application de la fonction de nettoyage à la colonne 'text'
print("🔄 Nettoyage en cours...")
df['cleaned_text'] = df['text'].apply(clean_text)

print("✅ Colonne 'cleaned_text' ajoutée")
print(f"\nExemple :")
print(f"  Original : {df.iloc[0]['text']}")
print(f"  Nettoyé  : {df.iloc[0]['cleaned_text']}")

🔄 Nettoyage en cours...
✅ Colonne 'cleaned_text' ajoutée

Exemple :
  Original : When Aeneas later traveled to Hades , he called to her ghost but she neither spoke to nor acknowledged him .
  Nettoyé  : when aeneas later traveled to hades he called to her ghost but she neither spoke to nor acknowledged him


## Étape 4 : Lemmatization

Ajout de la colonne `lemmatized_text` au dataset.

In [17]:
def lemmatize_text(text):
    """
    Lemmatise le texte en utilisant le modèle spaCy.
    La lemmatization transforme chaque mot en sa forme canonique (ex: 'running' -> 'run').
    
    Args:
        text (str): Le texte nettoyé à lemmatiser.
        
    Returns:
        str: Le texte lemmatisé (mots séparés par des espaces).
    """
    # Traitement du texte par le pipeline spaCy
    doc = nlp(text)
    
    # Extraction du lemme pour chaque token du document
    # token.lemma_ renvoie la forme lemmatisée du mot
    lemmas = [token.lemma_ for token in doc]
    
    # Reconstitution de la phrase sous forme de chaîne de caractères
    return ' '.join(lemmas)

# Application de la lemmatization à la colonne 'cleaned_text'
print("🔄 Lemmatization en cours (cela peut prendre quelques minutes)...")
df['lemmatized_text'] = df['cleaned_text'].apply(lemmatize_text)

print("✅ Colonne 'lemmatized_text' ajoutée")
print(f"\nExemple de texte lemmatisé :")
print(f"  {df.iloc[0]['lemmatized_text'][:100]}...")

🔄 Lemmatization en cours (cela peut prendre quelques minutes)...
✅ Colonne 'lemmatized_text' ajoutée

Exemple de texte lemmatisé :
  when aeneas later travel to hade he call to her ghost but she neither speak to nor acknowledge he...


## Étape 5 : Sauvegarde du Dataset Enrichi

In [18]:
# Vérification des colonnes présentes dans le DataFrame après enrichissement
print("📋 Colonnes du dataset enrichi :")
print(list(df.columns))
print(f"\nNombre de lignes : {len(df)}")

# Aperçu des premières lignes pour valider les nouvelles colonnes
df.head()

📋 Colonnes du dataset enrichi :
['id', 'words', 'ner_tags', 'text', 'cleaned_text', 'lemmatized_text']

Nombre de lignes : 700


,id,words,ner_tags,text,cleaned_text,lemmatized_text
0,en-doc5809-sent11,"['When' 'Aeneas' 'later' 'traveled' 'to' 'Hades' ',' 'he' 'called' 'to'\n 'her' 'ghost' 'but' 's...",[0 2 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],"When Aeneas later traveled to Hades , he called to her ghost but she neither spoke to nor acknow...",when aeneas later traveled to hades he called to her ghost but she neither spoke to nor acknowle...,when aeneas later travel to hade he call to her ghost but she neither speak to nor acknowledge he
1,en-doc6123-sent45,['On' '23' 'November' '1969' 'he' 'wrote' 'to' 'The' 'Times' 'newspaper'\n 'saying' 'that' 'the'...,[0 0 0 0 0 0 0 4 4 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0],On 23 November 1969 he wrote to The Times newspaper saying that the preparation for show trials ...,on 23 november 1969 he wrote to the times newspaper saying that the preparation for show trials ...,on 23 november 1969 he write to the times newspaper say that the preparation for show trial in c...
2,en-doc5831-sent40,"['Stephenson' ""'s"" 'estimates' 'and' 'organising' 'ability' 'proved' 'to'\n 'be' 'inferior' 'to'...",[2 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 2 0 0 0],Stephenson 's estimates and organising ability proved to be inferior to those of Locke and the b...,stephenson s estimates and organising ability proved to be inferior to those of locke and the bo...,stephenson s estimate and organise ability prove to be inferior to those of locke and the board ...
3,en-doc6189-sent73,['France' 'then' 'postponed' 'a' 'visit' 'by' 'Sharon' '.'],[1 0 0 0 0 0 2 0],France then postponed a visit by Sharon .,france then postponed a visit by sharon,france then postpone a visit by sharon
4,en-doc6139-sent18,"['Only' 'twenty-seven' 'years' 'old' 'at' 'his' 'death' ',' 'Moseley'\n 'could' 'in' 'many' 'sci...",[0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0],"Only twenty-seven years old at his death , Moseley could in many scientists ' opinions have cont...",only twenty seven years old at his death moseley could in many scientists opinions have contribu...,only twenty seven year old at his death moseley could in many scientist opinion have contribute ...


In [19]:
# Sauvegarde du dataset enrichi dans un nouveau fichier CSV
# index=False évite d'ajouter une colonne d'index inutile
output_file = 'data_preprocessed.csv'
df.to_csv(output_file, index=False)

print(f"✅ Dataset enrichi sauvegardé : {output_file}")
print(f"   Colonnes : {list(df.columns)}")
print(f"   Nombre de lignes : {len(df)}")

✅ Dataset enrichi sauvegardé : data_preprocessed.csv
   Colonnes : ['id', 'words', 'ner_tags', 'text', 'cleaned_text', 'lemmatized_text']
   Nombre de lignes : 700


## Résumé

### Dataset Enrichi

Le fichier `data_preprocessed.csv` contient maintenant **6 colonnes** :

**Colonnes originales (conservées) :**
1. `id` : Identifiant unique
2. `words` : Liste des mots tokenisés
3. `ner_tags` : Tags NER
4. `text` : Texte original

**Colonnes ajoutées :**
5. `cleaned_text` : Texte nettoyé (lowercase, sans caractères spéciaux)
6. `lemmatized_text` : Texte lemmatisé avec spaCy

### Utilisation (Partie B)

```python
import pandas as pd

# Charger le dataset enrichi
df = pd.read_csv('data_preprocessed.csv')

# Accéder aux différentes versions du texte
original_texts = df['text']
cleaned_texts = df['cleaned_text']
lemmatized_texts = df['lemmatized_text']

# Accéder aux annotations NER originales
ner_tags = df['ner_tags']
```

### Pipeline Appliqué

1. **Nettoyage** : Lowercase + Suppression caractères spéciaux + Normalisation espaces
2. **Lemmatization** : spaCy `en_core_web_sm`

**Note** : Toutes les colonnes originales sont conservées intactes !